In [3]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.optim import AdamW  # PyTorch의 AdamW 사용
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# 정제된 데이터 파일 경로 설정
output_excel_file = 'C:/Users/Main/Desktop/dataset/Dataset.xlsx'

# 데이터프레임으로 데이터 로드
df = pd.read_excel(output_excel_file)

# 텍스트 및 레이블 열 설정
text_column = 'Sentence'
label_column = 'Emotion'

# 학습 및 평가에 사용할 텍스트와 레이블 추출
sentences = df[text_column].values
labels = df[label_column].values

# 레이블 인코딩
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# 학습 및 검증 데이터로 분할
train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    sentences, labels, test_size=0.2, random_state=42
)

# 토크나이저 및 인코딩
tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-generator")
train_encodings = tokenizer(train_sentences.tolist(), truncation=True, padding=True, max_length=32, return_tensors='pt')
val_encodings = tokenizer(val_sentences.tolist(), truncation=True, padding=True, max_length=32, return_tensors='pt')

# 레이블 텐서로 변환
train_labels = torch.tensor(train_labels, dtype=torch.long)
val_labels = torch.tensor(val_labels, dtype=torch.long)

# 레이블 수 계산
num_labels = len(set(labels))

# 모델 로드
model = AutoModelForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-generator", num_labels=num_labels)

# 옵티마이저 설정 - PyTorch의 AdamW 사용
optimizer = AdamW(model.parameters(), lr=5e-5)

# 배치 크기 조정
batch_size = 128

# 모델 훈련
model.train()
for epoch in range(2):
    for i in range(0, len(train_labels), batch_size):
        optimizer.zero_grad()
        batch_train_encodings = {key: val[i:i+batch_size] for key, val in train_encodings.items()}
        batch_train_labels = train_labels[i:i+batch_size]
        outputs = model(**batch_train_encodings, labels=batch_train_labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step() 
    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

# 모델 저장 경로 설정
model_save_path = "C:/Users/Main/Desktop/dataset/koelectra_model.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-generator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Main\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Loss: 0.7252038717269897
Epoch 2, Loss: 0.6498829126358032
Model saved to C:/Users/Main/Desktop/dataset/koelectra_model.pth
